In [1]:
import os
from datetime import datetime

working_path = os.getcwd()
print(working_path)

if working_path == '/content':
    from google.colab import drive
    drive.mount('/content/drive')

    %cd /content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob

    path = "/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob/"

    # check python file folder
    assert os.path.exists(path + "tensorflow"), f"Upload python files in {path}tensorflow"
    %cd ./tensorflow

    # check data folder
    assert os.path.exists(path + "data"), f"Upload data files in {path}data"
else:
    path = "../"

    # check python file folder
    assert os.path.exists(path + "tensorflow"), f"Upload python files in {path}python"

    # check data folder
    assert os.path.exists(path + "data"), f"Upload data files in {path}data"

/content
Mounted at /content/drive
/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob
/content/drive/Othercomputers/Il mio MacBook Pro/MagicKnob/tensorflow


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

tfk = tf.keras
tfkl = tf.keras.layers

from model_utils import save_model

In [3]:
# used for the writing of example outputs
run_name="audio_ht1"
# dataset : need an input and output folder in this folder
audio_folder = f"../data/{run_name}"
#audio_folder = "../../data/audio_ht1"
assert os.path.exists(audio_folder), f"Audio folder  not found. Looked for {audio_folder}"
# used to render example output during training
test_file = "../data/guitar.wav"
assert os.path.exists(test_file), "Test file not found. Looked for " + test_file

In [4]:
# initialize net specs
lstm_hidden_size = 64
learning_rate = 5e-3
batch_size = 20 # già default

In [5]:
import myk_data
import myk_loss

input_audio_folder = f'{audio_folder}/input'
output_audio_folder = f'{audio_folder}/output'

In [6]:
[train_dataset, val_dataset] = myk_data.generate_dataset(input_audio_folder, output_audio_folder)

loading input and output of ht1
    loading output of ht1
    generate_dataset:: Loaded frames from audio file 2048
    found input fragments of shape (1291, 2048, 1)
    found output fragments of shape (1291, 2048, 1)
    total input shape: (1291, 2048, 1)
    total output shape: (1291, 2048, 1)



In [7]:
train_dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(2048, 1), dtype=tf.float32, name=None), TensorSpec(shape=(2048, 1), dtype=tf.float32, name=None))>

In [8]:
train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [9]:
train_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 2048, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2048, 1), dtype=tf.float32, name=None))>

In [10]:
def build_LSTM(input_shape, hidden_size):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    lstm = tfkl.LSTM(hidden_size, return_sequences=True)(input_layer)

    dense = tfkl.Dense(32, activation='linear')(lstm)

    output_layer = tfkl.Dense(1, activation='linear')(dense)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=myk_loss.LossWrapper(), optimizer=tfk.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])

    # Return the model
    return model

model = build_LSTM((None, 1), lstm_hidden_size)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, None, 1)]         0         
                                                                 
 lstm (LSTM)                 (None, None, 64)          16896     
                                                                 
 dense (Dense)               (None, None, 32)          2080      
                                                                 
 dense_1 (Dense)             (None, None, 1)           33        
                                                                 
Total params: 19,009
Trainable params: 19,009
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Train the model
history = model.fit(
    x = train_dataset,
    epochs = 500,
    validation_data = val_dataset,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, factor=0.5, min_lr=1e-5)
    ]
).history

Epoch 1/500
52/52 [==============================] - 13s 88ms/step - loss: 0.6946 - accuracy: 0.0107 - val_loss: 0.6441 - val_accuracy: 0.0093 - lr: 0.0050
Epoch 2/500
52/52 [==============================] - 4s 76ms/step - loss: 0.6820 - accuracy: 0.0107 - val_loss: 0.6462 - val_accuracy: 0.0093 - lr: 0.0050
Epoch 3/500
52/52 [==============================] - 3s 57ms/step - loss: 0.6492 - accuracy: 0.0107 - val_loss: 0.6517 - val_accuracy: 0.0093 - lr: 0.0050
Epoch 4/500
52/52 [==============================] - 3s 57ms/step - loss: 0.6613 - accuracy: 0.0107 - val_loss: 0.6972 - val_accuracy: 0.0093 - lr: 0.0050
Epoch 5/500
52/52 [==============================] - 4s 71ms/step - loss: 0.6775 - accuracy: 0.0107 - val_loss: 0.6469 - val_accuracy: 0.0093 - lr: 0.0050
Epoch 6/500
52/52 [==============================] - 3s 61ms/step - loss: 0.6539 - accuracy: 0.0107 - val_loss: 0.6395 - val_accuracy: 0.0093 - lr: 0.0050
Epoch 7/500
52/52 [==============================] - 3s 57ms/step - l

In [12]:
guitar = myk_data.load_wav_file(test_file, 44100)

In [13]:
guitar_dist = model.predict(guitar)

6176/6176 [==============================] - 12s 2ms/step


In [14]:
#guitar_dist = np.squeeze(guitar_dist, 2)
test = guitar_dist.flatten()
test = myk_data.normalize(test)
test

array([-0.23451255, -0.23416045, -0.23372744, ..., -0.2250875 ,
       -0.22475657, -0.22439055], dtype=float32)

In [15]:
import soundfile

soundfile.write("../data/guitar_dist.wav", test, 44100)

In [16]:
save_model(model, 'tensorflow_model2.json')

Skipping layer: <keras.engine.input_layer.InputLayer object at 0x7ac95ec21240>
